# 서울 공동주택 실거래가 예측

## Contents
- Library Import
- Data Load
- Data Preprocessing
- Feature Engineering
- Model Training
- Inference
- Output File Save

## 1. Library Import
- 필요한 라이브러리를 불러옵니다.

In [ ]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/font/NanumGothic.otf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from lightgbm import LGBMRegressor

import eli5
from eli5.sklearn import PermutationImportance

from datetime import datetime
import time
from zoneinfo import ZoneInfo
import os

## 2. Data Load

#### 2.1. 데이터 로드

In [ ]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = './data/train.csv'
test_path  = './data/test.csv'
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [ ]:
# Train data와 Test data shape은 아래와 같습니다.
print('Train data shape : ', train_df.shape, 'Test data shape : ', test_df.shape)

## 3. Data Preprocessing

- 모델링 전에 데이터 내 결측치, 이상치 등을 제거하고 범주형과 연속형 변수를 살펴보도록 하겠습니다!
- 먼저, 용이한 전처리를 위해 train과 test data를 합친 하나의 데이터로 진행하도록 하겠습니다.

In [ ]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
train_df['is_test'] = 0
test_df['is_test'] = 1
concat = pd.concat([train_df, test_df])     # 하나의 데이터로 만들어줍니다.

In [ ]:
concat['is_test'].value_counts()      # train과 test data가 하나로 합쳐진 것을 확인할 수 있습니다.

In [ ]:
# EDA를 통해 사용 결정한 필드들만 뽑아서 새로운 DataFrame 생성
selected_fields = [
    '시군구', '번지', '본번', '부번', '도로명', '아파트명',
    '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
    'target', 
    'is_test'
    ]

concat_selected_df = concat[selected_fields].copy()

In [ ]:
concat_selected_df = concat_selected_df.rename(columns={'전용면적(㎡)':'전용면적'})

In [ ]:
concat_selected_df

In [ ]:
concat_selected_df.info()

In [ ]:
concat_selected_df['본번'] = concat_selected_df['본번'].astype('str')
concat_selected_df['부번'] = concat_selected_df['부번'].astype('str')

concat_selected_df['target'] = concat_selected_df['target'].fillna(0)
concat_selected_df['target'] = concat_selected_df['target'].astype('int64')

concat_selected_df.info()

### 지하철 데이터 병합

In [ ]:
train_subway_path = './data/train_with_subway_infos.csv'
train_subway_df = pd.read_csv(train_subway_path)
print(train_subway_df.shape)
print(train_subway_df.info())
print(train_subway_df.head(3))

In [ ]:
test_subway_path  = './data/test_with_subway_infos.csv'
test_subway_df = pd.read_csv(test_subway_path)
print(test_subway_df.shape)
print(test_subway_df.info())
print(test_subway_df.head(3))

In [ ]:
total_subway_df = pd.concat([train_subway_df, test_subway_df], axis=0) 
print(total_subway_df.shape)
print(total_subway_df.info())
print(total_subway_df.head(3))

In [ ]:
concat_selected_df = pd.concat([concat_selected_df, total_subway_df], axis=1) 
print(concat_selected_df.shape)
print(concat_selected_df.info())
print(concat_selected_df.head(3))

### 버스 데이터 병합

In [ ]:
train_bus_path = './data/train_with_bus_infos.csv'
train_bus_df = pd.read_csv(train_bus_path)
print(train_bus_df.shape)
print(train_bus_df.info())
print(train_bus_df.head(3))

In [ ]:
test_bus_path  = './data/test_with_bus_infos.csv'
test_bus_df = pd.read_csv(test_bus_path)
print(test_bus_df.shape)
print(test_bus_df.info())
print(test_bus_df.head(3))

In [ ]:
total_bus_df = pd.concat([train_bus_df, test_bus_df], axis=0) 
print(total_bus_df.shape)
print(total_bus_df.info())
print(total_bus_df.head(3))

In [ ]:
# 지하철 정보에서 이미 머지되었으므로 버스에서는 삭제
del total_bus_df['좌표X_2']
del total_bus_df['좌표Y_2']

In [ ]:
concat_selected_df = pd.concat([concat_selected_df, total_bus_df], axis=1) 
print(concat_selected_df.shape)
print(concat_selected_df.info())
print(concat_selected_df.head(3))

## 5. Model Training

- 이제 위에서 만든 파생변수들과 정제한 데이터를 기반으로 본격적으로 부동산 실거래가를 예측하는 모델링을 진행하겠습니다.
- 모델링에는 LGBM을 이용하도록 하겠습니다.

### 5.1. 변수 Encoding

In [ ]:
continuous_features = []
categorical_features = []

for column in concat_selected_df.columns:
    if pd.api.types.is_numeric_dtype(concat_selected_df[column]):
        continuous_features.append(column)
    else:
        categorical_features.append(column)

continuous_features.remove('target')
continuous_features.remove('is_test')

print("연속형 변수:", continuous_features)
print("범주형 변수:", categorical_features)

#### 범주형 변수 인코딩

1. One-Hot Encoding
- 설명: 범주형 변수를 이진 벡터로 변환합니다. 각 범주는 고유한 이진 열을 가지며, 해당 범주에 속하면 1, 그렇지 않으면 0을 가집니다.
- 장점: 범주형 데이터의 모든 범주를 고유하게 표현합니다.
- 단점: 범주가 많을 경우 차원이 매우 커질 수 있습니다.

2. Label Encoding
- 설명: 각 범주를 고유한 정수로 변환합니다.
- 장점: 간단하고 메모리 효율적입니다.
- 단점: 범주 간의 순서가 암시될 수 있어 순서가 없는 명목형 변수에는 부적절할 수 있습니다.

3. Ordinal Encoding
- 설명: 순서가 있는 범주형 변수를 순서에 맞게 정수로 변환합니다.
- 장점: 순서형 변수에 적합합니다.

4. Frequency Encoding
- 설명: 각 범주를 해당 범주의 빈도로 변환합니다.
- 장점: 고유한 범주가 많을 때 유용합니다.

5. Target Encoding
- 설명: 각 범주를 해당 범주의 타겟 변수 평균으로 변환합니다.
- 장점: 특정 범주가 타겟 변수에 미치는 영향을 반영합니다.

[선택 기준]
- 고유 범주 수: 범주가 많을 경우 One-Hot Encoding은 차원이 커지므로 Frequency Encoding이나 Target Encoding이 더 적합할 수 있습니다.
- 변수의 특성: 순서가 있는 경우 Ordinal Encoding, 순서가 없는 경우 One-Hot Encoding이나 Frequency Encoding이 적합합니다.
- 모델의 특성: 트리 기반 모델(LightGBM, Random Forest 등)은 범주형 변수를 직접 처리할 수 있어 Label Encoding이 적합할 수 있습니다. 반면, 선형 모델에서는 One-Hot Encoding이 더 효과적일 수 있습니다

[추천]
- 부동산 예측 대회에서 자주 사용되는 범주형 변수 인코딩 방법은 One-Hot Encoding과 Target Encoding입니다.
- 트리 기반 모델을 사용하는 경우, Label Encoding도 좋은 선택이 될 수 있습니다. 

In [ ]:
# 범주형 변수 인코딩

# 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
label_encoders = {}

# Implement Label Encoding
for col in tqdm(categorical_features):
    lbl = LabelEncoder()

    # Label-Encoding을 fit
    lbl.fit(concat_selected_df[col].astype(str))
    concat_selected_df[col] = lbl.transform(concat_selected_df[col].astype(str))
    label_encoders[col] = lbl # 결과 분석시 원복을 위해 인코더를 저장

In [ ]:
for col in categorical_features:
    print(f"{col}: {concat_selected_df[col].shape}")

#### 연속형 변수 인코딩

1. StandardScaler
- 설명: 평균을 0, 분산을 1로 변환합니다.
- 장점: 데이터가 정규 분포를 따를 때 효과적입니다.

2. MinMaxScaler
- 설명: 데이터를 0과 1 사이로 스케일링합니다.
- 장점: 모든 피처의 값이 동일한 범위(0~1)로 변환되므로 비교적 큰 값과 작은 값의 차이를 줄여줍니다.

3. RobustScaler
- 설명: 중앙값(median)과 IQR(Interquartile Range)을 사용하여 스케일링합니다.
- 장점: 이상치(outlier)에 덜 민감합니다.

4. Log Transformation
- 설명: 로그 변환을 통해 비대칭적인 분포를 정규화합니다.
- 장점: 극단값의 영향을 줄이고 분포를 정규화합니다.

5. PowerTransformer
- 설명: 데이터의 분포를 더 정규 분포에 가깝게 변환합니다.
- 장점: 데이터가 강한 비대칭성을 가지고 있을 때 효과적입니다.

[선택 기준]
- 데이터 분포: 데이터의 분포가 정규 분포에 가까울 경우 StandardScaler가 적합하고, 비대칭성이 클 경우 PowerTransformer나 Log Transformation이 더 적합할 수 있습니다.
- 이상치(outlier): RobustScaler는 이상치에 덜 민감하므로, 이상치가 많을 경우 적합합니다.
- 해석 가능성: MinMaxScaler는 스케일링 후에도 데이터의 상대적인 순서를 유지하므로, 해석이 중요한 경우 유용할 수 있습니다.

[추천]
- 부동산 예측 대회에서 일반적으로 RobustScaler나 PowerTransformer가 좋은 선택이 될 수 있습니다.
- 이 둘은 이상치에 강하고 비대칭성을 처리하는 데 효과적이기 때문입니다.

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import PolynomialFeatures

# 연속형 변수 인코딩
scaler = RobustScaler()
concat_selected_df[continuous_features] = scaler.fit_transform(concat_selected_df[continuous_features])

# 4. 로그 변환 (양수 값에 대해서만 적용 가능)
# for feature in continuous_features:
#     if (concat_selected_df[feature] > 0).all():
#         concat_selected_df[f'{feature}_log'] = np.log1p(concat_selected_df[feature])

# # 5. 이산화 (예: '전용면적'을 구간으로 나누기)
# concat_selected_df['전용면적_구간'] = pd.cut(concat_selected_df['전용면적'], bins=5, labels=['매우작음', '작음', '중간', '큼', '매우큼'])

# # 6. 다항식 특성 (선택적)
# poly = PolynomialFeatures(degree=2, include_bias=False)
# poly_features = poly.fit_transform(concat_selected_df[continuous_features])
# poly_features_names = poly.get_feature_names_out(continuous_features)
# df_poly = pd.DataFrame(poly_features, columns=poly_features_names)
# df = pd.concat([concat_selected_df, df_poly], axis=1)

# 결과 확인
# print(df[continuous_features + [f'{feature}_log' for feature in continuous_features] + ['전용면적_구간']].head())
# print(df_poly.head())

In [ ]:
concat_selected_df.head(1)        # 인코딩이 된 모습입니다.

In [ ]:
# 이제 다시 train과 test dataset을 분할해줍니다. 위에서 제작해 놓았던 is_test 칼럼을 이용합니다.
dt_train = concat_selected_df.query('is_test==0')
dt_test = concat_selected_df.query('is_test==1')

# 이제 is_test 칼럼은 drop해줍니다.
dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)

dt_test.head(1)

### 5.2. Model Training
- 위 데이터를 이용해 모델을 train 해보겠습니다. 모델은 LightGBM 이용하겠습니다.
- Train과 Valid dataset을 분할하는 과정에서는 `holdout` 방법을 사용하겠습니다. 이 방법의 경우  대략적인 성능을 빠르게 확인할 수 있다는 점에서 baseline에서 사용해보도록 하겠습니다.
  - 이 후 추가적인 eda를 통해서 평가세트와 경향을 맞추거나 kfold와 같은 분포에 대한 고려를 추가할 수 있습니다.

In [ ]:
assert dt_train.shape[1] == dt_test.shape[1]          # train/test dataset의 shape이 같은지 확인해주겠습니다.

In [ ]:
# Target과 독립변수들을 분리해줍니다.
y_train = dt_train['target']
X_train = dt_train.drop(['target'], axis=1)

# Hold out split을 사용해 학습 데이터와 검증 데이터를 8:2 비율로 나누겠습니다.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2023)

In [ ]:
train_time = datetime.fromtimestamp(time.time(), tz=ZoneInfo("Asia/Seoul")).strftime("%Y%m%d-%H%M%S")
train_time

In [ ]:
# LightGBM 모델 정의
model = LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    num_leaves=31,
    random_state=1,
    n_jobs=-1
)

# 모델 학습
model.fit(X_train, y_train)

In [ ]:
model_file_path = os.path.join('model', f'{train_time}.pkl')

# 학습된 모델을 저장합니다. Pickle 라이브러리를 이용하겠습니다.
with open(model_file_path, 'wb') as f:
    pickle.dump(model, f)

#### Feature Importance

- 트리 기반 모델의 feature_importances_ 값을 확인하여 가장 중요도가 높은 특성을 식별합니다.
- 이 값들은 모델이 학습하는 과정에서 얼마나 자주 사용되었는지를 보여줍니다.
- 중요한 특성들을 기반으로 특성 선택이나 모델 튜닝을 수행할 수 있습니다.

In [ ]:
importance = model.feature_importances_
feature_names = model.feature_name_

sorted_features = sorted(zip(feature_names, importance), key=lambda x: x[1], reverse=True)
for name, importance in sorted_features:
    print(f'{name}: {importance}')

feature_names_series = pd.Series([x[0] for x in sorted_features], name='name')
importance_series = pd.Series([x[1] for x in sorted_features], name='importance')
plt.figure(figsize=(10,8))
plt.title("Feature Importances")
sns.barplot(x=importance_series, y=feature_names_series)
plt.show()

#### Permutation Importance

- Permutation Importance는 모델이 예측할 때 각 특성이 실제로 얼마나 중요한지를 평가합니다.
- 이를 통해 각 특성이 모델 예측의 정확도에 미치는 영향을 더 직관적으로 파악할 수 있습니다.

In [ ]:
# Permutation Importance 계산
perm = PermutationImportance(model,
                             scoring = 'neg_mean_squared_error', # 평가 지표로는 회귀문제이기에 negative rmse를 사용합니다. (neg_mean_squared_error : 음의 평균 제곱 오차)
                             random_state = 42,
                             n_iter=3).fit(X_val, y_val)

eli5.show_weights(perm, feature_names = X_val.columns.tolist())

In [ ]:
importance_df = eli5.explain_weights_df(perm, feature_names = X_val.columns.tolist())
plt.figure(figsize=(10, 8))
sns.barplot(x='weight', y='feature', data=importance_df.sort_values(by='weight', ascending=False))
plt.title('Permutation Importance')
plt.show()

### 5.3. 예측

In [ ]:
# 훈련 데이터에 대한 예측
y_train_pred = model.predict(X_train)

# 검증 데이터에 대한 예측
pred = model.predict(X_val)
pred = pred.astype(int) # 하락시기라서 일단 버림으로
pred

### 5.4. 평가 및 분석

- 훈련, 검증 세트에 대해 모델을 평가합니다.

In [ ]:
import import_ipynb
import evaluator

# get [description, rmse, r2, mae]
train_result = evaluator.evaluate_set(y_train, y_train_pred, "Train")
val_result = evaluator.evaluate_set(y_val, pred, "Valid")

comprehensive_report = evaluator.comprehensive_evaluation(train_result, val_result)
print(comprehensive_report)

In [ ]:
# 범주형 인코딩 원복 
for column in categorical_features :
    X_val[column] = label_encoders[column].inverse_transform(X_val[column])

In [ ]:
# 연속형 인코딩 원복
X_val[continuous_features] = scaler.inverse_transform(X_val[continuous_features])

In [ ]:
# 분석을 위해 Validation dataset에 target과 pred 값, 두 값의 사이의 오류치를 채워주도록 하겠습니다.
X_val['target'] = y_val
X_val['pred'] = y_val_pred

def calculate_error(target, pred):
    squared_errors = (target - pred)
    return squared_errors

squared_errors = calculate_error(X_val['target'], X_val['pred'])
X_val['error'] = squared_errors
X_val['abs_error'] = np.abs(X_val['error'])
X_val['error_rate'] = np.abs((X_val['target'] - X_val['pred']) / X_val['target'] * 100)

In [ ]:
print(f'valid rmse : {val_result[1]}')

threshold_absolute = val_result[1]  # 오차가 rmse 이상인 경우
threshold_relative = 50  # 오차율이 x% 이상인 경우
big_erros = X_val[(X_val['error'] >= threshold_absolute) & (X_val['error_rate'] >= threshold_relative)]
big_erros.sort_values(by='error_rate', ascending=False)

In [ ]:
error_row = train_df.iloc[199547]
error_row

## 6. Inference

In [ ]:
dt_test.head(2)      # test dataset에 대한 inference를 진행해보겠습니다.

In [ ]:
# 저장된 모델을 불러옵니다.
with open(model_file_path, 'rb') as f:
    model = pickle.load(f)

In [ ]:
%%time
X_test = dt_test.drop(['target'], axis=1)

# Test dataset에 대한 inference를 진행합니다.
real_test_pred = model.predict(X_test)

In [ ]:
real_test_pred          # 예측값들이 출력됨을 확인할 수 있습니다.

## 7. Output File Save

In [ ]:
# 앞서 예측한 예측값들을 저장합니다.
preds_df = pd.DataFrame(real_test_pred.astype(int), columns=["target"])
submission_file_path = os.path.join('output', f'{train_time}.csv')
preds_df.to_csv(submission_file_path, index=False)